In [4]:
import pandas as pd

This file aims to draft the replication of Commodities return following the KMPV methods.

In [2]:
raw_data = pd.read_parquet("../../_data/futures_returns/commodity_futures.parquet")

In [5]:
raw_data.head()

,index,KW1 Comdty_PX_LAST,KW2 Comdty_PX_LAST,KW3 Comdty_PX_LAST,SI3 Comdty_PX_LAST
0,1970-01-05,137.375,135.500,131.750,NaN
1,1970-01-06,138.250,136.750,132.625,NaN
2,1970-01-07,139.125,136.750,132.875,NaN
3,1970-01-08,139.500,137.250,133.125,NaN
4,1970-01-09,139.000,136.375,132.875,NaN


In [7]:
raw_data.to_csv("raw_data.csv")

KW -- Kansa Wheat  (Wheat in Yang's Paper)
SI -- Silver

In [ ]:
import pandas as pd
import numpy as np

df = raw_data.copy()

print(df.columns)

df['Date'] = pd.to_datetime(df['index'])
df = df.set_index('Date')

df_monthly = df.resample('M').last()


def compute_monthly_carry(df):
    maturities = np.array([1, 2, 3])

    price_cols = ['KW1 Comdty_PX_LAST', 'KW2 Comdty_PX_LAST', 'KW3 Comdty_PX_LAST']
    spot_list = []
    fut1m_list = []
    carry_list = []

    for _, row in df[price_cols].iterrows():
        prices = row.values.astype(float)
        coefs = np.polyfit(maturities, prices, 2)

        spot = np.polyval(coefs, 0)
        fut1m = np.polyval(coefs, 1)
        carry = (spot - fut1m) / fut1m

        spot_list.append(spot)
        fut1m_list.append(fut1m)
        carry_list.append(carry)

    df_carry = df.copy()
    df_carry['Spot_Price'] = spot_list
    df_carry['Futures_1M'] = fut1m_list
    df_carry['Carry'] = carry_list

    return df_carry

df_carry = compute_monthly_carry(df_monthly)


df_carry['Simple_Return'] = df_carry['KW1 Comdty_PX_LAST'].pct_change()

# Roll return
roll_prices = []
kw1 = df_carry['KW1 Comdty_PX_LAST']
kw2 = df_carry['KW2 Comdty_PX_LAST']
kw3 = df_carry['KW3 Comdty_PX_LAST']

for i in range(len(df_carry)):
    if i % 3 == 0:
        roll_prices.append(kw1.iloc[i])
    elif i % 3 == 1:
        roll_prices.append(kw2.iloc[i])
    else:
        roll_prices.append(kw3.iloc[i])

df_carry['Roll_Price'] = roll_prices
df_carry['Roll_Return'] = df_carry['Roll_Price'].pct_change()

print(df_carry[['Carry', 'Simple_Return', 'Roll_Return']])


Index(['index', 'KW1 Comdty_PX_LAST', 'KW2 Comdty_PX_LAST',
       'KW3 Comdty_PX_LAST', 'SI3 Comdty_PX_LAST'],
      dtype='object')


C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_30236\3482676097.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df.resample('M').last()


               Carry  Simple_Return  Roll_Return
Date                                            
1970-01-31  0.032374            NaN          NaN
1970-02-28  0.001787       0.006295    -0.008993
1970-03-31  0.098732      -0.013405    -0.019056
1970-04-30  0.097670       0.010870     0.032377
1970-05-31 -0.007729      -0.072581    -0.057348
...              ...            ...          ...
2025-02-28 -0.029557      -0.036254    -0.010790
2025-03-31 -0.019749      -0.002239     0.020942
2025-04-30 -0.030627      -0.076750    -0.120940
2025-05-31 -0.010783       0.036947     0.063685
2025-06-30 -0.009910       0.040788     0.085466

[666 rows x 3 columns]


In [18]:
df_carry

,index,KW1 Comdty_PX_LAST,KW2 Comdty_PX_LAST,KW3 Comdty_PX_LAST,SI3 Comdty_PX_LAST,Spot_Price,Futures_1M,Carry,Simple_Return,Roll_Price,Roll_Return
Date,,,,,,,,,,,
1970-01-31,1970-01-30,139.000,135.25,132.250,NaN,143.500,139.000,0.032374,NaN,139.000,NaN
1970-02-28,1970-02-27,139.875,137.75,133.750,NaN,140.125,139.875,0.001787,0.006295,137.750,-0.008993
1970-03-31,1970-03-31,138.000,132.50,135.125,NaN,151.625,138.000,0.098732,-0.013405,135.125,-0.019056
1970-04-30,1970-04-30,139.500,134.00,136.625,NaN,153.125,139.500,0.097670,0.010870,139.500,0.032377
1970-05-31,1970-05-28,129.375,131.50,134.750,NaN,128.375,129.375,-0.007729,-0.072581,131.500,-0.057348
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,2025-02-28,558.250,573.00,586.000,31.803,541.750,558.250,-0.029557,-0.036254,573.000,-0.010790
2025-03-31,2025-03-31,557.000,570.00,585.000,35.219,546.000,557.000,-0.019749,-0.002239,585.000,0.020942
2025-04-30,2025-04-30,514.250,529.50,544.250,33.124,498.500,514.250,-0.030627,-0.076750,514.250,-0.120940


In [21]:
import sys
from pathlib import Path

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# Plotting style
sns.set()

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Load project modules
from he_kelly_manela import pull_he_kelly_manela

# Load project settings
from settings import config

DATA_DIR = Path(config("DATA_DIR")) / "cds_returns"
HE_DATA_DIR = Path(config("DATA_DIR")) / "he_kelly_manela"

from he_kelly_manela import pull_he_kelly_manela
he_kelly = pull_he_kelly_manela.load_he_kelly_manela_all(data_dir=HE_DATA_DIR)

he_kelly.columns.str.contains("cmd").sum()


col_lst = ["yyyymm"]
for i in range(1, 10):
    col_lst.append(f"Commod_0{i}")
for i in range(10, 24):
    col_lst.append(f"Commod_{i}")

# %%
he_kelly_df = he_kelly[col_lst].dropna(axis=0)
he_kelly_df["Month"] = pd.to_datetime(
    he_kelly_df["yyyymm"].astype(int).astype(str), format="%Y%m"
)
he_kelly_df = he_kelly_df.drop(columns="yyyymm")
he_kelly_df = he_kelly_df.set_index("Month")
returns_pivot = df_carry
returns_pivot.index = returns_pivot.index.rename('Month')

he_kelly_df.index = he_kelly_df.index.to_period('M')
returns_pivot = returns_pivot["1986-09":"2012-12"]
corr_matrix = pd.DataFrame(index=returns_pivot.columns, columns=he_kelly_df.columns)


for col1 in returns_pivot.columns:
    for col2 in he_kelly_df.columns:
        x = returns_pivot[col1].fillna(0)
        y = he_kelly_df[col2].fillna(0)
        corr_matrix.loc[col1, col2] = x.corr(y)

corr_matrix_float = corr_matrix.astype(float)